### Compute results

In [7]:
import pandas as pd
import numpy as np
import nibabel as nib

from nilearn import datasets, image, plotting
from neurolang.frontend import NeurolangPDL

import neurolang
neurolang.config.disable_expression_type_printing()

JULICH_REGIONS_PATH = 'JULICH_BRAIN_CYTOARCHITECTONIC_MAPS_2_9_MNI152_2009C_NONL_ASYM.txt'
JULICH_ATLAS_PATH = 'JULICH_BRAIN_CYTOARCHITECTONIC_MAPS_2_9_MNI152_2009C_NONL_ASYM.pmaps.nii.gz'
PEAKS_PATH = 'peaks_IBC.csv'

julich = False
RESAMPLE = 1

/Users/gzanitti/.local/lib/python3.8/site-packages/nilearn-0.8.1-py3.8.egg/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


## Decoding

In [16]:
mni_t1 = nib.load(datasets.fetch_icbm152_2009()['t1'])
mni_t1_4mm = image.resample_img(mni_t1, np.eye(3) * RESAMPLE)

if julich:

    lines = []
    with open(JULICH_REGIONS_PATH) as f:
        lines = f.readlines()

    count = 0
    res = []
    for line in lines:
        if count == 0:
            count += 1
            continue
        splited = line.split(' ')
        res.append((splited[0], ' '.join(splited[1:-1])[1:], splited[-1][:-2]))

    regions = pd.DataFrame(res, columns=['r_number', 'r_name', 'hemis'])
    regions = regions[~regions.r_name.str.contains('GapMap')]
    regions = regions.astype({'r_number': 'int64'})
    region_number = 'r_number'
    region_name = 'r_name'

    pmaps_4d = image.resample_img(
        image.load_img(JULICH_ATLAS_PATH), mni_t1_4mm.affine, interpolation='nearest'
    )

    brain_regions_prob = []
    prob_region_data = pmaps_4d.dataobj
    non_zero = np.nonzero(pmaps_4d.dataobj)
    for x, y, z, r in zip(*non_zero):
        p = prob_region_data[x, y, z, r]
        d = (p, x, y, z, r)
        brain_regions_prob.append(d)

else:
    difumo_components = 256
    brain_regions_prob = pd.read_hdf('./atlas_difumo.hdf', key=f'data_{difumo_components}')[['weights', 'i', 'j', 'k', 'component']]
    regions = pd.read_hdf('./atlas_difumo.hdf', key=f'labels_{difumo_components}')
    region_number = 'Component'
    region_name = 'Difumo_names'



# IBC data
ibc = pd.read_csv(PEAKS_PATH)
ibc = ibc[ibc['Cluster Size (mm3)'] > 20]

ibc = ibc[['subject_id', 'img_id', 'X', 'Y', 'Z', 'Peak Stat', 'Cluster Size (mm3)']]

ijk_positions = (
    nib.affines.apply_affine(
        np.linalg.inv(mni_t1_4mm.affine),
        ibc[['X', 'Y', 'Z']]
    ).astype(int)
)
ibc['i'] = ijk_positions[:, 0]
ibc['j'] = ijk_positions[:, 1]
ibc['k'] = ijk_positions[:, 2]

ibc = ibc[['subject_id', 'img_id', 'i', 'j', 'k', 'Peak Stat', 'Cluster Size (mm3)']]

In [42]:
import os
directory = './results/min_sample_difumo_1fold_out/'
 
df = pd.DataFrame([])
for filename in os.listdir(directory):
    path = directory + filename
    if os.path.isfile(path):
        temp = pd.read_hdf(path)
        df = pd.concat((df, temp))

df = df.sort_values(['region', 'fold'])

f_term = pd.read_hdf('./term2cp.hdf')

df = df.join(f_term.set_index('t'), on='term')

In [48]:
df.head()

,term,fold,region,p,pn,bf,cp
0,action perception,0,1,0.001943,0.019628,0.098973,
1,animacy decision,0,1,0.009678,0.034236,0.282703,Reasoning and Decision Making
2,animacy perception,0,1,0.009678,0.034236,0.282703,Perception
3,audiovisual perception,0,1,0.001943,0.019628,0.098973,Perception
4,auditory arithmetic processing,0,1,0.015365,0.010979,1.399510,


In [52]:
probtr = df.groupby(['term', 'region','cp']).mean().drop('fold', axis=1).reset_index()
probtr.head()

,term,region,cp,p,pn,bf
0,action perception,1,,0.001945,0.019472,0.099910
1,action perception,2,,0.002178,0.019909,0.109422
2,action perception,4,,0.017472,0.015854,1.102308
3,action perception,6,,0.008432,0.017298,0.487550
4,action perception,8,,0.018322,0.015844,1.156595


In [53]:
nl = NeurolangPDL()

j_brain = nl.add_tuple_set(
    #(prob, i, j, k, region)
    brain_regions_prob,
    name='julich_brain'
)

dataIBC = nl.add_tuple_set(
    #(subject, img, i, j, k, peak_stat, cluster_size)
    ibc,
    name='dataIBC'
)

ptr = nl.add_tuple_set(
    #(term, region, cp, p, bf)
    probtr,
    name='prob_term_region'
)

KeyboardInterrupt: 

In [39]:
image_terms = {}
final_probs = {}

import os
directory = './results/min_sample_difumo_1fold_out/'
 
df = pd.DataFrame([])
for filename in os.listdir(directory):
    path = directory + filename
    if os.path.isfile(path):
        temp = pd.read_hdf(path)
        df = pd.concat((df, temp))

df = df.sort_values(['region', 'fold'])

f_term = pd.read_hdf('./term2cp.hdf')

df = df.join(f_term.set_index('t'), on='term')

validation_images = pd.read_csv('./removed_min_img.csv')
for img in validation_images.img_id.values[:5]:
    final_probs = {}
    for term in df.term.unique():
        if julich:
            aud_sen = df[df.term == term].groupby('region').mean().join(regions.set_index(region_number)).reset_index()[['region', region_name, 'hemis', 'p', 'pn', 'bf']]
        else:
            aud_sen = df[df.term == term].groupby('region').mean().join(regions.set_index(region_number)).reset_index()[['region', region_name, 'p', 'pn', 'bf']]
        res_image = res[res.image == img]
        joined = aud_sen[['region', 'p']].set_index('region').join(res_image.set_index('region'), lsuffix='_term').fillna(0)
        joined['prob'] = joined['p_term'] * joined['p']
        term_prob = joined.prob.sum()
        final_probs[term] = term_prob

    image_terms[img] = final_probs

In [40]:
resdf = pd.DataFrame([])

ibc_tag = pd.read_csv(PEAKS_PATH)
ibc_tag = ibc_tag[ibc_tag['Cluster Size (mm3)'] > 20]

for image, terms_probs in image_terms.items():
    tmpdf = pd.DataFrame(terms_probs.items(), columns=['term', 'prob'])
    tmpdf['img_id'] = image
    tags = ibc_tag[ibc_tag.img_id == image].tag.unique()
    if len(tags) == 1:
        tmpdf['ibc_tags'] = tags[0]
    else:
        print(f'Error tag in {image}')
    resdf = pd.concat((resdf, tmpdf))

In [41]:

for img in validation_images.img_id.values:
    rmpdf = resdf[resdf.img_id == img].sort_values('prob', ascending=True)
    print(img)
    print(rmpdf.ibc_tags.values[0])
    print(rmpdf.head(10)[['term', 'prob']].values)
    print()

367372
action perception,  audiovisual perception, auditory scene analysis, social cognition
[['response execution' 4.4930070270248725]
 ['response selection' 5.485083612892996]
 ['auditory perception' 13.216631965931347]
 ['working memory' 13.401458586773584]
 ['reading' 19.416595746852863]
 ['sentence processing' 24.169352466593136]
 ['visual face recognition' 28.89443547744922]
 ['spatial working memory' 31.142431312165073]
 ['visual word recognition' 31.64488033667359]
 ['theory of mind' 32.53603741919091]]

367689
speech perception,  speech processing,  language comprehension,  language processing,  action perception,  audiovisual perception,  auditory scene analysis,  social cognition
[['response execution' 3.8236037907270024]
 ['response selection' 4.649465007639732]
 ['working memory' 11.520049476495455]
 ['auditory perception' 12.315333334590589]
 ['reading' 16.570040796753815]
 ['sentence processing' 21.839747435383984]
 ['visual face recognition' 24.69425848360634]
 ['featur

IndexError: index 0 is out of bounds for axis 0 with size 0

## Mono-Multi Modal

In [6]:
import nilearn
from nilearn import datasets, image, plotting
import nibabel as nib
import numpy as np
import pandas as pd

RESAMPLE = 1

mni_t1 = nib.load(datasets.fetch_icbm152_2009()['t1'])
mni_t1_4mm = image.resample_img(mni_t1, np.eye(3) * RESAMPLE)

def get_difumo_img(mask, n_components=256, out_dir='./difumo', resolution='3mm'):

    DIFUMO_N_COMPONENTS_TO_DOWNLOAD_ID = {
        64: 'pqu9r',
        128: 'wjvd5',
        256: '3vrct',
        512: '9b76y',
        1024: '34792',
        }

    download_id = DIFUMO_N_COMPONENTS_TO_DOWNLOAD_ID[n_components]
    url = f"https://osf.io/{download_id}/download"
    csv_file = os.path.join(
        str(n_components), f"labels_{n_components}_dictionary.csv"
    )
    nifti_file = os.path.join(str(n_components), "%s/maps.nii.gz"%resolution)
    files = [
        (csv_file, url, {"uncompress": True}),
        (nifti_file, url, {"uncompress": True}),
    ]
    files = nilearn.datasets.utils._fetch_files(out_dir, files, verbose=2)
    labels = pd.read_csv(files[0])
    img = nilearn.image.load_img(files[1])
    img = nilearn.image.resample_to_img(
        img,
        target_img=mask,
        interpolation="nearest",
    )

    return img, labels

difumo_img, difumo_labels = get_difumo_img(mni_t1_4mm)

In [7]:
import os
directory = './results/min_sample_difumo_1fold_out/'
 
df = pd.DataFrame([])
for filename in os.listdir(directory):
    path = directory + filename
    if os.path.isfile(path):
        temp = pd.read_hdf(path)
        df = pd.concat((df, temp))

df = df.sort_values(['region', 'fold'])

f_term = pd.read_hdf('./term2cp.hdf')

df = df.join(f_term.set_index('t'), on='term')

In [8]:
def calc_MIP(image, axis=0):
    data = image.get_fdata()
    ndata = np.maximum.reduce(data, axis=axis)
    img = nib.Nifti1Image(ndata, image.affine)

    return img

def calc_argMIP(image, axis=0):
    data = image.get_fdata()
    ndata = data.argmax(axis=axis)
    img = nib.Nifti1Image(ndata, image.affine)

    return img

pmaps_3d = calc_argMIP(difumo_img, axis=3)
plotting.plot_img(pmaps_3d)

In [ ]:
data3 = pmaps_3d.get_fdata()

non_zero = np.nonzero(data3)
rmap = np.zeros_like(data3)

dfterm = df[df.term == 'short term memory'].groupby('region').mean().reset_index()[['region', 'bf']]
dict_bfs = dict(zip(dfterm.region, dfterm.bf))

for x, y, z in zip(*non_zero):
    reg = int(data3[x, y, z])
    if reg in dict_bfs:
        rmap[x, y, z] = dict_bfs[reg]

In [ ]:
with nl.scope as e:


    '''e.reg_prob[e.region, e.image, e.PROB[e.region, e.image]] = (
        (
            e.julich_brain[..., e.i, e.j, e.k, e.region]
        ) // (
            e.dataIBC[..., e.image, e.i, e.j, e.k, ..., ...]
        )
    )'''

    e.complement_regions[e.region, e.complement] = (
        e.prob_term_region[..., e.region, ..., ..., ...] &
        e.prob_term_region[..., e.complement, ..., ..., ...] &
        (e.complement != e.region)
    )

    '''e.complement_prob[e.term, e.prod(e.inv_region)] = (
        e.prob_term_region[e.term, e.region, e.topconcept, e.p, e.pn] &
        (e.inv_region = )
    )

    e.decoding[e.term, e.region] = (
        e.reg_prob[e.region, e.image, e.prob_region]
    )'''
            
    res = nl.query((e.region, e.complement), e.complement_regions[e.region, e.complement])
    res = res.as_pandas_dataframe()